# Capstone Project- Data Science


In [40]:
import pandas as pd
import numpy as np
import requests
!pip install folium
import folium
# !pip install wikipedia
# import wikipedia as wiki
# !pip install geopy.extra
import matplotlib.cm as cm
import matplotlib.colors as colors
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim

tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


In [4]:
print('Hello Capstone project Course')

Hello Capstone project Course


# TriCity Poland

My project looks at grouping similar living areas across the various districts in the TriCity.

In [5]:
# District names take from Wikipideda

gdansk_districts=['Osowa','Oliwa','Żabianka, Jelitkowo','Przymorze Małe','Przymorze Wielkie','VII Dwór','Strzyża',
                  'Zaspa-Młyniec','Zaspa-Rozstaje','Brzeźno','Matarnia','Brętowo','Wrzeszcz Dolny','Wrzeszcz Górny',
                  'Letnica','Nowy Port','Piecki-Migowo','Suchanino','Siedlce','Wzgórze Mickiewicza','Aniołki','Młyniska',
                  'Stogi','Przeróbką','Śródmieście','Krakowiec-Górki Zachodnie','Wyspa Sobieszewska','Kokoszki','Chełm',
                  'Ujeścisko-Łostowice','Jasień','Orunia-Św. Wojciech-Lipce','Olszynka','Rudniki']

sopot_districts=['Sopot Centrum','Karlikowo','Kamienny Potok','Przylesie','Dolny Sopot Grunwaldzka','Dolny Sopot Haffnera',
                 'Lisie Wzgórze','Zajęcze Wzgórze','Górny Sopot','Brodwino','Osiedle Mickiewicza','Świemirowo','Stawowie',
                 'Leśna Polana\Gręzowo','Centrum Południe – Kościuszki','Sopocki Las']

gdynia_districts=['Babie Doły', 'Chwarzno-Wiczlino', 'Chylonia', 'Cisowa', 'Dąbrowa', 'Działki Leśne', 'Grabówek', 
                  'Kamienna Góra', 'Karwiny', 'Leszczynki', 'Mały Kack', 'Obłuże', 'Oksywie', 'Orłowo', 'Pogórze', 'Pustki Cisowskie-Demptowo', 
                  'Redłowo', 'Śródmieście', 'Wielki Kack', 'Witomino-Leśniczówka', 'Witomino-Radiostacja', 'Wzgórze Św. Maksymiliana']

df=pd.DataFrame()

# for i in range(len(gdynia_districts)):
#     gdynia_districts[i]=gdynia_districts[i]+', Gdynia'
    

# for i in range(len(sopot_districts)):
#     sopot_districts[i]=sopot_districts[i]+', Sopot'
    
# for i in range(len(gdansk_districts)):
#     gdansk_districts[i]=gdansk_districts[i]+', Gdansk'
    

df_gdansk=pd.DataFrame(columns=['District'])
df_sopot=pd.DataFrame(columns=['District'])
df_gdynia=pd.DataFrame(columns=['District'])

In [6]:
df_gdansk['District']=gdansk_districts
df_sopot['District']=sopot_districts
df_gdynia['District']=gdynia_districts

In [7]:
df_sopot.head()

,District
0,Sopot Centrum
1,Karlikowo
2,Kamienny Potok
3,Przylesie
4,Dolny Sopot Grunwaldzka


In [8]:
df=df.append(sopot_districts)
df=df.append(gdansk_districts)
df=df.append(gdynia_districts)

df.rename(columns={0:'Districts'},inplace=True)    

df.reset_index(drop=True,inplace=True)



In [9]:
df['Latitude']='default'
df['Longitude']='default'

In [10]:
# k=[]
# for i in range(13, len(df['Districts'])):
#     address=df['Districts'][i]+', Poland'
#     location=None
#     while location is None :
#             geolocator=Nominatim(user_agent='Capstone')
#             location=geolocator.geocode(address)
#     df['Latitude'][i]=location.latitude
#     df['Longitude'][i]=location.longitude


In [13]:
# df.drop([23,56],inplace=True)
# df.reset_index(drop=True,inplace=True)

df.head()

,Districts,Latitude,Longitude
0,Sopot Centrum,54.441369,18.562259
1,Karlikowo,54.737308,18.149749
2,Kamienny Potok,54.457252,18.529620
3,Przylesie,54.324033,17.948903
4,Dolny Sopot Grunwaldzka,54.444025,18.569975


In [14]:
#Download and save the file for futre use in case the geopy doesn't work

from IPython.display import HTML
import base64  
import pandas as pd  

def create_download_link( df, title = "Download CSV file", filename = "final_data.csv"):  
    csv = df.to_csv(index =False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(df)

In [15]:
map_tri = folium.Map(location=[54.3920, 18.5407], zoom_start=10)

# add markers to map
for lat, lng, district, in zip(df['Latitude'], df['Longitude'], df['Districts']):
    label = '{}'.format(district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tri)  
    
map_tri

In [16]:
# FourSquare Credentials:



In [53]:
def getNearbyVenues(names, latitudes, longitudes, radius=400):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
#         print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [54]:
tricity_venues = getNearbyVenues(names=df['Districts'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )
tricity_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Sopot Centrum,54.441369,18.562259,Cyrano et Roxane,54.443063,18.561148,French Restaurant
1,Sopot Centrum,54.441369,18.562259,"SEAFOOD STATION RESTAURANT, BAR & GRILL",54.441943,18.562558,Seafood Restaurant
2,Sopot Centrum,54.441369,18.562259,Sopot,54.440254,18.562524,Train Station
3,Sopot Centrum,54.441369,18.562259,Whiskey on the rocks,54.441499,18.562620,Lounge
4,Sopot Centrum,54.441369,18.562259,Cesky Film,54.443253,18.562254,Czech Restaurant


In [55]:
# one hot encoding
tricity_onehot = pd.get_dummies(tricity_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tricity_onehot['Neighborhood'] = tricity_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tricity_onehot.columns[-1]] + list(tricity_onehot.columns[:-1])
tricity_onehot = tricity_onehot[fixed_columns]

tricity_onehot.head()

,Neighborhood,American Restaurant,Art Gallery,Asian Restaurant,Auto Garage,Automotive Shop,Bakery,Bar,Bay,Beach,...,Theme Park,Thrift / Vintage Store,Track Stadium,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Yoga Studio,Zoo Exhibit
0,Sopot Centrum,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Sopot Centrum,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Sopot Centrum,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,Sopot Centrum,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Sopot Centrum,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
tricity_grouped = tricity_onehot.groupby('Neighborhood').mean().reset_index()
tricity_grouped.head()

,Neighborhood,American Restaurant,Art Gallery,Asian Restaurant,Auto Garage,Automotive Shop,Bakery,Bar,Bay,Beach,...,Theme Park,Thrift / Vintage Store,Track Stadium,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Yoga Studio,Zoo Exhibit
0,Aniołki,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Babie Doły,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Brodwino,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Brzeźno,0.0,0.0,0.0,0.2,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Brętowo,0.0,0.0,0.0,0.0,0.666667,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [62]:
# CLustering 

# set number of clusters
kclusters = 4

tricity_grouped_clustering = tricity_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tricity_grouped_clustering)

kmeans.labels_

array([0, 2, 1, 0, 2, 2, 2, 0, 2, 0, 2, 2, 3, 1, 2, 2, 2, 2, 0, 2, 1, 2, 2,
       0, 1, 2, 2, 2, 1, 2, 2, 1, 1, 0, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 0, 0,
       2, 1, 2, 3, 2], dtype=int32)

In [63]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tricity_grouped['Neighborhood']

for ind in np.arange(tricity_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tricity_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aniołki,Hotel,Italian Restaurant,Pizza Place,Park,Diner,Fast Food Restaurant,Farm,Electronics Store,Eastern European Restaurant,Donut Shop
1,Babie Doły,Trail,Zoo Exhibit,Diner,Food,Fast Food Restaurant,Farm,Electronics Store,Eastern European Restaurant,Donut Shop,Dessert Shop
2,Brodwino,Modern European Restaurant,Bus Station,Liquor Store,Seafood Restaurant,Grocery Store,Zoo Exhibit,Eastern European Restaurant,Fast Food Restaurant,Farm,Electronics Store
3,Brzeźno,Food & Drink Shop,Hotel,Auto Garage,Boarding House,Grocery Store,Eastern European Restaurant,Food,Fast Food Restaurant,Farm,Electronics Store
4,Brętowo,Automotive Shop,Garden Center,Donut Shop,Food & Drink Shop,Food,Fast Food Restaurant,Farm,Electronics Store,Eastern European Restaurant,Diner


In [64]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tricity_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tricity_merged = tricity_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Districts')

tricity_merged.head() # check the last columns!

,Districts,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Sopot Centrum,54.441369,18.562259,2.0,Café,Sushi Restaurant,Pizza Place,Bistro,Burger Joint,Seafood Restaurant,Breakfast Spot,Food,Lounge,French Restaurant
1,Karlikowo,54.737308,18.149749,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Kamienny Potok,54.457252,18.529620,1.0,Seafood Restaurant,Liquor Store,Bus Station,Grocery Store,Zoo Exhibit,Fast Food Restaurant,Farm,Electronics Store,Eastern European Restaurant,Donut Shop
3,Przylesie,54.324033,17.948903,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dolny Sopot Grunwaldzka,54.444025,18.569975,2.0,Café,Hotel,Seafood Restaurant,Bar,Italian Restaurant,Sushi Restaurant,Plaza,Beach,Eastern European Restaurant,Dessert Shop


In [65]:
tricity_merged.dropna(axis=0,inplace=True)
tricity_merged['Cluster Labels']=tricity_merged['Cluster Labels'].astype(int)

In [66]:
# create map
map_clusters = folium.Map(location=[54.3920, 18.5407], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tricity_merged['Latitude'], tricity_merged['Longitude'], tricity_merged['Districts'], tricity_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters